Objasnjenje koda u prvoj code sekciji. Preuzima se HTML kod sa neke stranice i pretvara se u BeautifulSoup objekat.

In [282]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from bs4 import BeautifulSoup
from pathlib import Path 
start_url = "https://www.loudersound.com/features/the-50-greatest-guitarists-of-all-time"
chromedriver_location=r"C:\Users\Korisnik\miniconda3\Scripts\chromedriver.exe"
service = Service(chromedriver_location)
def get_soup(url):
    driver = webdriver.Chrome()
    driver.get(url) # driver za chrome
    return BeautifulSoup(driver.page_source, features='html.parser') # atribut source page drivera + opcioni features
    
soup = get_soup(start_url)

2. Metoda get_specific_page() polazi od start_url-a i vraca URL sa brojem zeljene stranice koja je uneta groz parametar.

In [140]:
def get_specific_page(start_url,page=1):
    if page>1:
        return start_url+'/'+str(page)
    else:
        return start_url
print(get_specific_page(start_url, 4))

https://www.loudersound.com/features/the-50-greatest-guitarists-of-all-time/4


3. Metoda vraca supu sa zadate stranice. 

In [276]:
def get_next_soup(start_url,page_number=1):
    return get_soup(get_specific_page(start_url,page_number)) 
soup = get_next_soup(start_url,2)

In [277]:
print(soup.text)





The 50 Greatest Guitarists Of All Time 40-31 - The 50 best guitarists of all time | Louder














































































































 











Skip to main content





Open menu
Close menu







Louder












Louder Together we're Louder





Search

 


Search Louder



 



Subscribe



 

 

 

 

 





Music MagazinesWhy subscribe?SUBSCRIBE AND GET YOUR NEXT 3 ISSUES FOR JUST £5! Every issue delivered direct to your doorOr treat someone you love to the gift that keeps on giving!From$27.25View







 



NEWS




FEATURES




REVIEWS




STORE






 



 



 



 



 





TrendingNew BMTH singleJonathan Davis talks sexAce Frehley vs KISSMetal on late-night TV

When you purchase through links on our site, we may earn an affiliate commission. Here’s how it works.






Features


Louder



The 50 Best Guitarists Of All Time


By Louder published September 29, 2018 

The 50 best guitarists of all ti

4. Metoda find_text_info() koja ima bs objekat i vraca listu uredjenih cetvorki za svakog autora u obliku (name, rank, author, band)

In [289]:
def find_next_info(soup):
    result_list = list()
    names_and_ranks_of_gitarist =soup.findAll('h2')
    for elem in names_and_ranks_of_gitarist[:-1]: # poslednji je Get the Louder Newsletter
       rank_and_name = elem.text.split('.')
       rank = int(rank_and_name[0])
       name = rank_and_name[1].strip()
       #print(rank, name)
       author_group = elem.find_next('strong').text.split(" (")
       author = author_group[0]
       group = author_group[1].split("):")[0].split(")")[0] if len(author_group)==2 else None
       #print(group)
       result_list.append((name, rank, author, group))
    return result_list  

list_of_tuples =find_next_info(soup)
for elem in list_of_tuples:
    print(elem)

('Robert Fripp', 50, 'Steven Wilson', None)
('John McLaughlin', 49, 'Chris Goss', 'Producer/Masters Of Reality')
('Steve Morse', 48, 'Steve Howe', 'Yes')
('Peter Green', 47, 'Rich Robinson', 'The Black Crowes, The Magpie Salute')
('Robin Trower', 46, 'Paul Gilbert', 'Racer X, Mr Big')
('Tom Morello', 45, 'Wayne Kramer', 'MC5')
('Paul Gilbert', 44, 'Jim Davies', 'The Prodigy, Pitchshifter')
('Malcolm Young', 43, 'Scott Ian', 'Anthrax')
('George Harrison', 42, 'Elliot Easton', 'The Cars')
('Michael Schenker', 41, 'Mike McReady', 'Pearl Jam')


5. Kreiranje metode get_youtube_video. Ona polazi od bs objekta i rednog broja odgovarajuce YouTube videa i vraca link na video

In [290]:
def get_youtube_video(soup:BeautifulSoup,video_number:int):
    gitarists = soup.findAll('h2')
    for gitarist in gitarists[:-1]:
        rank,_=gitarist.text.split(".") #if len(gitarist.text.split(". ") == 2)  
        #print(rank)
        #print(rank)
        rank_int = int(rank)  
        if rank_int != video_number:
            continue
        else:
            yt_link=gitarist.find_next('div',attrs={'class':'youtube-video'}).find('div',attrs={'class':"video-aspect-box"}).find_next('iframe').get('data-lazy-src')
            #print(yt_link)
            yt_base, rest = yt_link.split("/embed")
            res = yt_base+ "watch?v="+rest
            #print(yt_base+'watch?v='+rest)
        
            return res 
            
#yt_link=get_youtube_video(soup,48) # ovde je primio supu prvu ispisao 50-41 i None zbog 31 jer on nije na prvoj stranuci
#print(yt_link)

6. Polazi od bs objekta i vraca za taj objekat listu linkova na tom objektu.

In [291]:
def get_yt_links(soup1):
    guitarists = soup1.find_all('h2')
    list_of_links=list()
    for guitarist in guitarists[:-1]:
        rank, _ = guitarist.text.split(".")  # 48.&nbsp;Steve Morse - greska je da ja delim kao ". " umesto "." zbog ovog elementa 48
        rank_int = int(rank)
        yt_link = get_youtube_video(soup, rank_int)
        #print(f"{rank}:{yt_link}")
        list_of_links.append(yt_link)
    
    return list_of_links
#sp = get_next_soup(start_url,2)
links = get_yt_links(soup)
for link in links:
    print(link)

https://www.youtube.comwatch?v=/Lm8xCYZjB-M
https://www.youtube.comwatch?v=/OMTXjmVd6AI
https://www.youtube.comwatch?v=/U609XEuKLPc
https://www.youtube.comwatch?v=/RtmW2ek7WkQ
https://www.youtube.comwatch?v=/sTpvd3UAXP4
https://www.youtube.comwatch?v=/3L4YrGaR8E4
https://www.youtube.comwatch?v=/rn-wj4pRpIE
https://www.youtube.comwatch?v=/eQluGs2SFRs
https://www.youtube.comwatch?v=/R1TLj9-jdLs
https://www.youtube.comwatch?v=/JjPW35eot70


7. crawl() polazi od start url-a,  ide do maximalnog broja stranica  i radi web crawling. Neka je crawl generatorska metoda. 

In [292]:
def crawl(start_url, max_pages=1): 
    for page in range(max_pages): 
        yield get_next_soup(start_url,page+1) #za max pages = 2, gleda range 0,1 ali u yield poziva 1 i 2.
soups = crawl(start_url,3)
while True:
    try:
        print(next(soups))
    except StopIteration:
        break
        

<html class="" data-locale="GB" dir="ltr" lang="en"><link href="//vanilla.futurecdn.net/loudersound/854159/media/css/loudersound.min.css" rel="stylesheet" type="text/css"/><head class="">
<!-- [METATAGS - critical] -->
<meta charset="utf-8"/>
<meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
<title>The 50 best guitarists of all time | Louder</title>
<meta content="2018-09-29T09:54:30+00:00" name="pub_date"/>
<meta content="The 50 best guitarists of all time, as voted by you" name="description"/>
<link href="https://www.loudersound.com/features/the-50-greatest-guitarists-of-all-time" rel="canonical"/>
<meta content="max-image-preview:large" name="robots"/>
<link data-before-rewrite-localise="https://www.loudersound.com/features/the-50-greatest-guitarists-of-all-time/2" href="https://www.loudersound.com/features/the-50-greatest-guitarists-of-all-time/2" rel="next"/>
<link data-before-rewrite-localise="https://www.loudersound.com/features/the-50-greatest

8. get_article_info_list() metoda prolazi kroz urla pocetne stranice, do maximalnog broja i vraca listu uredjenih petorki (name, rank, autor, band)

In [275]:
def get_article_info_list(start_url,max_pages=1):
    all_soups = crawl(start_url,max_pages)
    res_list = []
    while True:
        try:
            soup = next(all_soups)
            list_of_information=find_next_info(soup)
            list_of_links = get_yt_links(soup)
            for i in range(len(list_of_information)):
                name, rank, author, band = list_of_information[i]
                print(name,rank,author,band)
                link = list_of_links[i]
                print(link)
                res_list.append((name, rank, author, band, link))
        except StopIteration:
            break 
    return res_list

list_r = get_article_info_list(start_url,3)
# for el in list_r:
#     print(el)
        

Robert Fripp 50 Steven Wilson None
https://www.youtube.comwatch?v=/Lm8xCYZjB-M
John McLaughlin 49 Chris Goss Producer/Masters Of Reality
https://www.youtube.comwatch?v=/OMTXjmVd6AI
Steve Morse 48 Steve Howe Yes
https://www.youtube.comwatch?v=/U609XEuKLPc
Peter Green 47 Rich Robinson The Black Crowes, The Magpie Salute
https://www.youtube.comwatch?v=/RtmW2ek7WkQ
Robin Trower 46 Paul Gilbert Racer X, Mr Big)
https://www.youtube.comwatch?v=/sTpvd3UAXP4
Tom Morello 45 Wayne Kramer MC5
https://www.youtube.comwatch?v=/3L4YrGaR8E4
Paul Gilbert 44 Jim Davies The Prodigy, Pitchshifter
https://www.youtube.comwatch?v=/rn-wj4pRpIE
Malcolm Young 43 Scott Ian Anthrax)
https://www.youtube.comwatch?v=/eQluGs2SFRs
George Harrison 42 Elliot Easton The Cars
https://www.youtube.comwatch?v=/R1TLj9-jdLs
Michael Schenker 41 Mike McReady Pearl Jam)
https://www.youtube.comwatch?v=/JjPW35eot70
Duane Allman 40 Charlie Starr Blackberry Smoke)
None
Paul Kossoff 39 Paul Mahon The Answer)
None
Keith Richards 38 Stev

9. csv za datu listu. 

In [293]:
import csv
from sys import stderr 
def write_into_csv(list_of_results):
    try:
        with open(Path.cwd()/'../data/top50.csv','w') as fobj:
            csv_writer = csv.writer(fobj)
            csv_writer.writerow(('Singer','Rank','Author','Band','Link'))
            csv_writer.writerows(list_of_results)
    except OSError as err:
        stderr.write(f"OS ERROR.\n{err}\n")
        
write_into_csv(list_r)